In [21]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YouTubeContentRepository
from use_case import YouTubeScriptRefinement

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
repository = YouTubeContentRepository(client)

# LLM
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# 유즈-케이스
youtube_script_refinement = YouTubeScriptRefinement(repository, openai_llm)

contents = repository.find_all()

content = contents[-1]

Connected to MongoDB


# 아웃 파서

In [22]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# ✅ (타임라인 섹션 모델)
class YoutubeTimelineSectionModel(BaseModel):
    title: str = Field(..., description="섹션 제목")
    sec: float = Field(..., description="섹션 시작 시간 (초 단위)")
    texts: List[str] = Field(..., description="해당 섹션의 자세한 내용 리스트")
    tip: str = Field(..., description="해당 섹션에서 AI의 추가적인 팁")

# ✅ (타임라인 요약 모델)
class YoutubeTimelineSummaryModel(BaseModel):
    description: str = Field(..., description="전체 영상 설명 요약")
    sections: List[YoutubeTimelineSectionModel] = Field(..., description="타임라인별 섹션 리스트")

# ✅ 아웃파서 정의
timeline_summary_parser = PydanticOutputParser(pydantic_object=YoutubeTimelineSummaryModel)

# 프롬프트 템플릿

In [23]:
from langchain.prompts import PromptTemplate

timeline_summary_prompt_template = PromptTemplate(
    partial_variables={'output_format': timeline_summary_parser.get_format_instructions()},
    input_variables=["title", "description", "script"],
    template="""
당신은 주어지는 유튜브 정보를 다른 사람에게 설명하는 전문가입니다.

다음은 유튜브 영상의 설명과 스크립트입니다.

[유튜브 영상 제목]:
{title}

[유튜브 영상 설명]:
{description}

[대본 스크립트]:
{script}

[타임라인 섹션 예시: 식기 세척기 구매 고민 해결 영상]
- 전체 내용 설명: 이 영상은 식기세척기 구매에 대한 중요한 정보를 제공합니다. LG, 삼성, SK매직의 제품들을 비교 분석하며, 각 브랜드의 주요 기능과 사용자 편의성을 강조합니다. 특히, 가성비와 성능에 따라 적합한 모델을 선택하는 방법에 대한 팁을 제공합니다. 소비자는 자신의 설치 환경과 취향에 맞춰 최적의 선택을 할 수 있도록 도움을 받을 수 있습니다. 이 영상은 궁극적으로 식기세척기구매에서 성공적인 선택을 돕기 위한 것입니다

- 섹션1 주제: 🧼식기세척기 용량 및 브랜드 비교
- 식기세척기의 설치 타입은 빌트인과 프리스탠딩으로 나뉘며, 각 타입의 장단점을 고려하여 설치 환경에 맞게 선택할 필요가 있다.
- LG 식기세척기는 스팀 기능 유무에 따라 기본형(3종), 중급형(6종), 고급형(6종)으로 나뉘며, 상부 터치모델이 가성비가 좋다고 알려져 있다.
- 삼성제품은 제조 연도에 따라 기능 차이가 있으며, 기본형 라인은 60에서 100만원대의 가격으로 높은 가성비를 제공하고 있다.
- 추천 제품은 매월 최신화되므로, 소비자는 노서치를 통해 확인할 수 있다. [1-6]

- 섹션2 주제: 🍽️브랜드별 식기세척기 기능 비교
- LG는 100도씨 스팀 기능을 통해 기름기 있는 음식물을 효과적으로 세척하고, 물 얼룩을 방지하는 연수장치를 탑재하여 위생성을 높였다
- 삼성은 열풍 건조 기능을 3단계로 조절 가능하며, 비스포크 디자인으로 인테리어와의 통일성을 강조하고, 모든 제품이 빌트인 프리스탠딩겸용으로 출시되어 설치 편의성을 제공한다.
-  SK매직은 자동 문닫힘기능과 UV램프를 통해 세척과 건조 효율을 높이며, 상 중하 3개의 세척날개와 추가 분사구를 통해 사각지대까지 물을 분사할 수 있는 특징이 있다.
- 각 브랜드는 세척 성능과 용량의 업그레이드는 더 이상 큰 변화가 없을 것으로 보며, 수납 효율, 건조 방법, 디자인을 통한 경쟁이 예상된다.

- 섹션3 주제: 🛒브랜드별 추천 제품 정리
- 주어진 브랜드별 핵심 기능을 바탕으로 두세 가지의 제품 라인업을 추천했다.
- 대안으로 고려할 수 있는 제품도 함께 정리하였다.

- 섹션4 주제: 🏡식기세척기 설치 타입과 브랜드별 추천
- 식기세척기의 설치 타입은 빌트인과 프리스탠딩으로 나뉘며, 각 타입의 장단점을 고려하여 설치 환경에 맞게 선택할 필요가 있다.
- LG 식기세척기는 스팀 기능 유무에 따라 기본형(3종), 중급형(6종), 고급형(6종)으로 나뉘며, 상부 터치모델이 가성비가 좋다고 알려져 있다.
- 삼성제품은 제조 연도에 따라 기능 차이가 있으며, 기본형 라인은 60에서 100만원대의 가격으로 높은 가성비를 제공하고 있다.
- 식기세척기의 설치 타입은 빌트인과 프리스탠딩으로 나뉘며, 각 타입의 장단점을 고려하여 설치 환경에 맞게 선택할 필요가 있다.
- SK매직은 다양한 모델을 제공하며, 터치홈플러스와 터치원 프로 라인의 기능과 가격을 비교하여 선택할 수 있으며, 프리미엄 라인은 자동 문 닫힘 기능과 함께 편리함을 제공한다.

- 섹션5 주제: 🛒브랜드별 추천 식기세척기 모델
- 가성비를 중점적으로 고려한다면, LG의 dubj1p, 삼성의 dw60t 7075, SK매직의 dwa-91c를 추천한다.
- 브랜드의 특징과 기능을 중요시한다면, LG의 dubea, 삼성의 dw60a8355가 적합하다.
- 모든 기능을 사용할 수 있는 프리미엄 제품이 필요하다면, LG의 dubj사일, 삼성의 dw60 bb8376, SK매직의 dwa90c를 추천한다.
- 이 정보가 식기세척기선택에 도움이 되었는지 확인하고, 유용한 다른 영상도 참고하라고 제안한다.

이 정보를 바탕으로 영상의 타임라인 기반 요약을 생성해주세요.

- **전체 내용을 설명해주는 요약을 작성**하세요.
- **영상 내용을 타임라인별로 나누어 설명하세요.
- 각 타임라인 제목 앞에 제목에 어울리는 이모지를 붙여주세요.
- **각 타임라인 섹션은 `제목(title)`, `시작 시간(sec)`, `내용(texts)`, `팁(tip)` 을 포함**해야 합니다.
- 섹션 별 내용(texts) 개수는 최대한 많이 만들어야 합니다.
- 설명된 내용만 보고도 따라할 수 있고, 정확한 정보를 얻을 수 있어야 합니다.
- 단계 별로 안내할 경우 각 단계의 모든 설명을 작성해주세요.
- 사용자는 영상을 보지 않아도 충분히 인사이트를 얻어갈 수 있어야 합니다.
- 사용자는 완전한 초보자라고 가정합니다. 스크립트에서 추상적으로 설명하거나 빠르게 스킵한 경우 최대한 자세하게 과정을 임의로 적어주세요.
- **[유튜브 영상 설명]을 참고하여 더 자연스럽고 매끄럽게 작성하세요.
- AI의 추가적인 tip도 작성해주세요. 이때 AI는 해당 주제의 전문가입니다. 최대한 많이 알려주려고 하세요.
- **반드시 아래 JSON 형식에 맞춰 반환**하세요. 다른 텍스트는 포함하지 마세요.
- 출력 토큰을 아끼지마세요.

{output_format}
"""
)

# LLM

In [24]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [25]:
script_refine_chain = timeline_summary_prompt_template | openai_llm | timeline_summary_parser

input_data = {
        "title": content.title,
        "description": content.description,
        "script": "\n".join(
            [f"({int(chunk.start_time)}-{int(chunk.end_time)}): {chunk.text}" for chunk in content.script.chunks]
        )
    }

response: YoutubeTimelineSummaryModel = script_refine_chain.invoke(input_data)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [26]:
print(response)

description='이 영상에서는 Flyweight 패턴이라는 메모리 최적화 패턴을 소개합니다. 대량의 인스턴스가 공유 리소스를 통해 메모리와 성능을 효율적으로 최적화하는 방법에 대해 설명합니다. 특히 Unity에서 스크립터블 오브젝트를 활용하여 Flyweight 패턴을 구현하는 방법에 대해 자세히 알아보세요.' sections=[YoutubeTimelineSectionModel(title='🛠️ Flyweight 패턴 소개', sec=0.0, texts=['Flyweight 패턴은 대량의 인스턴스를 메모리에서 최적화하는 패턴입니다.', '객체 복제로 인한 메모리 낭비를 방지하기 위해 이 패턴을 사용할 수 있습니다.', '여러 객체가 리소스를 공유하여 효율적으로 메모리를 활용할 수 있도록 돕습니다.'], tip='Flyweight 패턴은 특정 상황에서 메모리 절약을 통해 성능 향상을 가져올 수 있습니다. 대량의 객체를 다루는 경우 이 패턴을 고려해 보세요.'), YoutubeTimelineSectionModel(title='📊 Flyweight 패턴의 필요성', sec=18.0, texts=['Flyweight 패턴은 메모리 사용량을 줄여주는 데 초focus되어 있습니다.', '이 패턴을 사용하면 코드 유지 보수도 간결해지는 이점이 있습니다만, 메모리가 가장 중요한 포커스입니다.', '여러 비슷한 객체들이 존재할 때, 각 객체가 중복된 데이터를 갖지 않고 공통된 데이터를 참조할 수 있도록 돕습니다.'], tip='메모리 최적화 필요성을 느낄 때 Flyweight 패턴을 도입하면 좋습니다. 특히 큰 게임 리소스를 관리할 때 유용합니다.'), YoutubeTimelineSectionModel(title='✈️ 비행기를 예로 들어보자', sec=93.0, texts=['다수의 비행기 객체를 통해 Flyweight 패턴의 예시를 살펴볼 수 있습니다.', '각 비행기는 공통적으로 공유해야 하는 데이터를 가진 채 여러 인스턴스가 존재할 수 있습니다.', '비행기의 색

In [27]:
from domain import YoutubeTimelineSummary, YoutubeTimelineSection

sections = [
    YoutubeTimelineSection(
        title=section.title,
        sec=section.sec,
        texts=section.texts,
        tip=section.tip
    )
    for section in response.sections
]

timeline_summary = YoutubeTimelineSummary(text=response.description, sections=sections)

In [28]:
print(timeline_summary)

YoutubeTimelineSummary(text='이 영상에서는 Flyweight 패턴이라는 메모리 최적화 패턴을 소개합니다. 대량의 인스턴...', sections=[(0.0s: 🛠️ Flyweight 패턴...), (18.0s: 📊 Flyweight 패턴의...), (93.0s: ✈️ 비행기를 예로 들어보자...), ...] (6 sections))


In [29]:
content.set_timeline_summary(timeline_summary)

In [30]:
repository.save(content)

True